## Code fourni pour lire les binaires:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time

def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]

def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]

def read_double_tab(f, n):
    ba = bytearray(8*n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm

def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    #print("Ouverture du fichier de pics "+filename)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    #print("Nb pics par trame: " + str(info["nb_pics"]))
    info["freq_sampling_khz"] = read_double(f_pic)
    #print("Frequence d'echantillonnage: " + str(info["freq_sampling_khz"]) + " kHz")
    info["freq_trame_hz"] = read_double(f_pic)
    #print("Frequence trame: " + str(info["freq_trame_hz"]) + " Hz")
    info["freq_pic_khz"] = read_double(f_pic)
    #print("Frequence pic: " + str(info["freq_pic_khz"]) + " kHz")
    info["norm_fact"] = read_double(f_pic)
    #print("Facteur de normalisation: " + str(info["norm_fact"]))
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames+1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    #print("Nb trames: " + str(nb_trames))
    f_pic.close()
    return tab_pics, info

    pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")
    pics_pad0, info = get_pics_from_file("../data/pics_0.bin")


    ######### Pics ############
    # NO KEY
    plt.figure(1)
    plt.subplot(211)
    plt.plot(range(1,info["nb_pics"]+1), np.mean(pics_nokey,axis=0), 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('no key')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    # PAD-0
    plt.subplot(212)
    plt.plot(range(1,info["nb_pics"]+1), pics_nokey[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('PAD-0')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    #
    plt.show()

# Notre Algorithme
* Enlever le bruit en faisant - Nokey ------- Inutile ?
* Séparer les trames en clusters ------- While == précédent + [un certain pas]
* Attention à shift+a par exemple --- Maybe in top best letters ?
* ctrl alt suppr au début ---- Histoire de pas oublier

### Bibliothèques utilisées :

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score

### Constitution du dataset

In [82]:
X,Y = [],[]

# Faut itérer maintenant
pics_A, info = get_pics_from_file("../data/pics_A.bin")
pics_B, info = get_pics_from_file("../data/pics_B.bin")
pics_C, info = get_pics_from_file("../data/pics_C.bin")

for trame in pics_A:
    X.append(trame)
    Y.append("A")
for trame in pics_B:
    X.append(trame)
    Y.append("B")
for trame in pics_C:
    X.append(trame)
    Y.append("C")

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

# Scaling des données
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### KNN Algorithm

In [83]:
# Best value for N (internet said this)
import math
N = math.floor(math.sqrt(len(Y_test)))
if N % 2 == 0:
    N -= 1

# KNN Clustering
classifier = KNeighborsClassifier(n_neighbors=N, p=2, metric='euclidean')
classifier.fit(X_train,Y_train)
Y_pred = classifier.predict(X_test)
Y_pred

array(['C', 'C', 'B', ..., 'B', 'C', 'C'], dtype='<U1')

### Evaluate Model

In [87]:
cm = confusion_matrix(Y_test,Y_pred)
# La diagonale sont les bons guess et le reste les fautes
print(cm)
print(accuracy_score(Y_test,Y_pred),"%")

[[1512   13    8]
 [  18 1509    3]
 [  25    5 1512]]
0.9843648208469056 %
